In [3]:
import pandas as pd
import datetime

In [4]:
actors = pd.read_csv('../Tables/Actors.csv')
directors = pd.read_csv('../Tables/Directors.csv')
genres = pd.read_csv('../Tables/Genres.csv')
movies = pd.read_csv('../Tables/Movie_Data.csv')
studios = pd.read_csv('../Tables/Studios.csv')
titles = pd.read_csv('../Tables/titleNid.csv')

In [5]:
actor_mean = actors.groupby('movie_id', as_index=False, sort = False).mean()
actor_mean = actor_mean.drop(['actor_id','gender'],axis=1)
actor_mean.to_csv('../Tables/actor_mean.csv', index=False)

In [6]:
director_mean = directors.groupby('movie_id', as_index=False , sort = False).mean()
director_mean = director_mean.drop(['director_id','gender'],axis=1)
director_mean.to_csv('../Tables/director_mean.csv', index=False)


In [7]:
count = studios.studio_name.value_counts()
replace = list(count[count <= 3].index)
replace
for studio in replace:
    studios.studio_name = studios.studio_name.replace(studio, "Other")
grouped_df = studios.groupby('movie_id',sort = False)
grouped_lists = grouped_df['studio_name'].agg(lambda column: ", ".join(column))
grouped_lists = grouped_lists.reset_index(name='studio_name')
holder = []
for names in grouped_lists['studio_name']:
    holder.append(names.split(","))
# movies['studio_list'] = holder
# movies
grouped_lists['studio_list'] = holder
grouped_lists = grouped_lists.drop('studio_name', 1)
grouped_lists.to_csv('../Tables/studio_assm.csv', index=False)


In [8]:
movies['studios'] = grouped_lists['studio_list']
movies['release_date'] = pd.to_datetime(movies['release_date'], format= '%Y-%m-%dT')
genres.set_index(genres['id'],inplace = True)
genres = genres.drop('id',1)

genre_title = []

for ls in movies.genre_ids:
    holder = []
    for id in eval(ls):
        holder.append(genres.loc[id]['name'])
    genre_title.append(holder)

movies['genres'] = genre_title
movies = movies.drop('genre_ids',1)
movies.to_csv('../Tables/movies_clean.csv', index=False)

In [9]:
movies.head()

,movie_id,title,release_date,movie_popularity,vote_average,vote_count,budget,revenue,rating,studios,genres
0,566525,Shang-Chi and the Legend of the Ten Rings,2021-09-01,5884.885,7.9,3428,150000000,430238384,PG-13,[Marvel Studios],"[Action, Adventure, Fantasy]"
1,580489,Venom: Let There Be Carnage,2021-09-30,5797.863,7.0,2452,110000000,454000000,PG-13,"[Marvel Entertainment, Pascal Pictures, Colu...","[Science Fiction, Action, Adventure]"
2,370172,No Time to Die,2021-09-29,3366.389,7.6,2075,242000000,734000000,PG-13,"[Other, Metro-Goldwyn-Mayer, Other, Univers...","[Adventure, Action, Thriller]"
3,524434,Eternals,2021-11-03,1746.171,7.1,1170,200000000,336000000,PG-13,[Marvel Studios],"[Action, Adventure, Science Fiction]"
4,744275,After We Fell,2021-09-01,1710.038,7.2,956,14000000,19000000,R,"[Other, Other, Other, Other, Other]","[Romance, Drama]"


In [10]:
ml_table = movies[['movie_id','release_date','movie_popularity', 'vote_average', 'vote_count','budget','revenue', 'rating', 'genres','studios']]
ml_table = ml_table.merge(actor_mean,on='movie_id', how = 'inner')
ml_table = ml_table.merge(director_mean,on='movie_id', how = 'inner')
ml_table = ml_table.drop('movie_id',1)
ml_table

,release_date,movie_popularity,vote_average,vote_count,budget,revenue,rating,genres,studios,actor_popularity,director_popularity
0,2021-09-01,5884.885,7.9,3428,150000000,430238384,PG-13,"[Action, Adventure, Fantasy]",[Marvel Studios],6.738895,5.119
1,2021-09-30,5797.863,7.0,2452,110000000,454000000,PG-13,"[Science Fiction, Action, Adventure]","[Marvel Entertainment, Pascal Pictures, Colu...",4.363038,13.077
2,2021-09-29,3366.389,7.6,2075,242000000,734000000,PG-13,"[Adventure, Action, Thriller]","[Other, Metro-Goldwyn-Mayer, Other, Univers...",4.164382,4.167
3,2021-11-03,1746.171,7.1,1170,200000000,336000000,PG-13,"[Action, Adventure, Science Fiction]",[Marvel Studios],5.096703,2.397
4,2021-09-01,1710.038,7.2,956,14000000,19000000,R,"[Romance, Drama]","[Other, Other, Other, Other, Other]",4.204667,2.580
...,...,...,...,...,...,...,...,...,...,...,...
69,2015-04-18,192.039,6.8,1246,5000000,61768190,NR,"[Action, Animation, Science Fiction]","[Other, Other, Other]",3.765667,0.600
70,2021-08-25,188.391,6.3,615,25000000,77389310,R,"[Horror, Thriller]","[Metro-Goldwyn-Mayer, Other, Other, Univers...",1.431400,0.600
71,2020-08-14,186.863,7.7,2351,60000000,4700000,PG,"[Family, Animation, Fantasy, Adventure, Comedy]","[Other, Other, Other, Other, Other, Other]",6.898650,2.717
72,2012-01-19,185.102,6.1,3312,79000000,355692760,PG,"[Adventure, Action, Science Fiction]","[Other, New Line Cinema, Other]",11.495923,1.425


In [12]:
ml_table.dtypes

release_date           datetime64[ns]
movie_popularity              float64
vote_average                  float64
vote_count                      int64
budget                          int64
revenue                         int64
rating                         object
genres                         object
studios                        object
actor_popularity              float64
director_popularity           float64
dtype: object